# Comparsion of the EM and MM method

This notebook demonstrates the performance of the EM algorithm proposed in the paper, where relevant results can be found in Table 1, Section 5.1.1.

## Preload packages and functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from scripts.estimation import *
from scripts.utils import *

# Generate Synthetic Data

We generate the synthetic data from a Poisson process with an underlying MNL model as rider's choice probability. In the MNL model, the intercept is `beta0` and the slope regarding the distance to an available bike is `beta1`. The arrival times of riders follow Poisson process with rate `lambd`. The number of generated arrival locations is set as `num_position`. The number of bikes that is included in the system is set as `bike_num`. We observe the process for in time period `[0,T]`. In this example, all arrival locations are generated randomly at the intersections of a 5x5 Cartesian grid.

In [2]:
rand_seed = 1
num_position = 5
bike_num = 30
lambd = 10
grid_size = 5
beta0 = 1
beta1_true = -1
T = 100
loc_bound = 5
true_pos_ind,bike_num,num_records,book_bike,book_index, \
dist,bike_loc,all_period,num_booked, \
cand_loc,true_loc,position_weight = gen_sync_in_grid(rand_seed,num_position,bike_num,lambd,
                                                     grid_size,beta0=beta0,beta1_true=beta1_true,T=T,loc_bound=loc_bound)

# Test on the EM algorithm
The following snippet tests the performance of the EM algorithm. All arrival locations are randomly generated in a Cartesian grid that is chosen as the candidate location set for the all-in algorithm. To test the accuracy of the EM algorithm, we select the first `num_position` locations in the candidate set decreasing ordered by the predicted weights. We compute the log-likelihood value and the Wasserstein distance between the predicted weights and the underlying truth.

In [3]:
num_position_true = num_position
num_position = grid_size**2

w = np.random.uniform(0,1,num_position)
w = w/np.sum(w)
w = w.reshape(1,-1)
diff_w = np.inf
beta1 = np.repeat(-1,num_position).reshape(-1,1)
dist_old = caldist(cand_loc[-1],bike_loc,bike_num)
iter = 0
threshold = 1e-4
while diff_w > threshold:
    iter = iter+1
    p_old = np.zeros((num_position,bike_num+1,num_records))
    p_olds = np.zeros((num_position,num_booked))
    w_old = w[-1,:]
    p_old[:,0,:] = 1/(1+np.sum(np.exp(beta0+np.reshape(beta1,(-1,1,1))*dist_old),axis=2))
    p_old[:,1:,:] = np.exp(beta0+np.reshape(beta1,(-1,1,1))*np.transpose(dist_old,(0,2,1)))/ \
        np.reshape((1+np.sum(np.exp(beta0+np.reshape(beta1,(-1,1,1))*dist_old),axis=2)),(num_position,1,-1))
    p_olds = p_old[:,book_bike+1,book_index]*np.reshape(w_old,(-1,1))/np.reshape(np.sum(np.reshape(w_old,(-1,1))*p_old[:,book_bike+1,book_index],axis=0),(1,-1))
    p_oldsj0 = w_old*np.sum(p_old[:,0,:]*all_period,axis=1)/np.sum(w_old*np.sum(p_old[:,0,:]*all_period,axis=1))
    s_old = np.sum((1-np.sum(np.expand_dims(w_old,1)*(1/(1+np.sum(np.exp(beta0+np.reshape(beta1,(-1,1,1))*dist_old),axis=2))),axis=0))*all_period)
    N_oldy = num_booked*(T-s_old)/s_old
    c = np.sum(p_olds,axis=1)+N_oldy*p_oldsj0
    w_star = np.reshape(c/np.sum(c),(1,-1))
    w = np.concatenate((w,w_star),axis=0)
    diff_w = np.sum(np.abs(w[-1,:]-w[-2,:]))

pre_w = w[-2]
true_w = np.zeros(grid_size**2)
true_w[true_pos_ind] = position_weight
sel_ind = np.where(pre_w>=np.sort(pre_w)[-num_position_true])[0]
true_pred_ind = np.intersect1d(sel_ind,true_pos_ind)
l1norm = np.sum(np.abs(pre_w-true_w))

thres = 0.01
sel_ind = pre_w>thres
num_remains = np.sum(sel_ind)
w_trim = pre_w[sel_ind]/np.sum(pre_w[sel_ind])
was_dist_disc = find_wasserstein(cand_loc[0,sel_ind,:],true_loc,w_trim,position_weight)
trimmed_lkd = findlkd_no_constraint(np.sum(sel_ind),caldist(cand_loc[0,sel_ind,:],bike_loc,bike_num),\
                                    beta0,np.repeat(beta1_true,np.sum(sel_ind)).reshape(-1,1),w_trim,
                                    bike_num,num_records,book_bike,book_index,num_booked,all_period)
bic = -trimmed_lkd+0.5*num_remains*np.log(num_booked)
print('Number of Iterations:', iter)
print('The Wasserstein distance is:',was_dist_disc[0])
print('The log-likelihood value is:',trimmed_lkd)
print('The BIC value is:', bic)

Number of Iterations: 478
The Wasserstein distance is: 3.2366596595842063
The log-likelihood value is: -3353.364043421947
The BIC value is: 3381.1272917181586


# Test on the MM algorithm

The following snippet tests the performance of the EM algorithm. Similarly, the number of iterations, the log-likelihood value, and the Wasserstein distance are reported to compare the MM algorithm to the EM algorithm.

In [4]:
def find_mm_sol(num_position,num_booked,s_k,choice_prob,all_period):
  w_new = cp.Variable((num_position,1))
  prob = cp.Problem(cp.Maximize(-num_booked/s_k * cp.sum(cp.multiply((1-cp.sum(cp.multiply(w_new,choice_prob[:,0,:]),axis=0)),all_period)) + \
      cp.sum(cp.log(cp.sum(cp.multiply(w_new,(choice_prob[:,book_bike+1,book_index])),axis=0)))),
      [cp.sum(w_new)==1,0<=w_new,w_new<=1]
  )
  result = prob.solve(solver=cp.SCS)
  return w_new.value


In [5]:
num_position = grid_size**2
w = np.random.uniform(0,1,num_position)
w = w/np.sum(w)
w = w.reshape(1,-1)
beta1 = np.repeat(-1,num_position).reshape(-1,1)
loc = cand_loc

dist1 = caldist(loc,bike_loc,bike_num)
choice_prob = np.zeros((num_position,bike_num+1,num_records))
choice_prob[:,0,:] = 1/(1+np.sum(np.exp(beta0+beta1.reshape(-1,1,1)*dist1),axis=2))
choice_prob[:,1:,:] = np.exp(beta0+beta1.reshape(-1,1,1)*np.transpose(dist1,(0,2,1)))/(1+np.sum(np.exp(beta0+beta1.reshape(-1,1,1)*dist1),axis=2)).reshape(num_position,1,-1)
diff_w = np.inf
threshold = 1e-4
iter = 0
while diff_w > threshold:
  iter = iter + 1
  w_old = w[-1]
  s_k = np.sum((1-np.sum(w_old.reshape(-1,1)*choice_prob[:,0,:],axis=0))*all_period)
  w_new = find_mm_sol(num_position,num_booked,s_k,choice_prob,all_period).reshape(1,-1)
  w = np.concatenate((w,w_new),axis=0)
  diff_w = np.sum(np.abs(w[-1]-w[-2]))

pre_w = w[-2]
true_w = np.zeros(grid_size**2)
true_w[true_pos_ind] = position_weight
sel_ind = np.where(pre_w>=np.sort(pre_w)[-num_position_true])[0]
true_pred_ind = np.intersect1d(sel_ind,true_pos_ind)
l1norm = np.sum(np.abs(pre_w-true_w))

thres = 0.01
sel_ind = pre_w>thres
num_remains = np.sum(sel_ind)
w_trim = pre_w[sel_ind]/np.sum(pre_w[sel_ind])
was_dist_disc = find_wasserstein(cand_loc[0,sel_ind,:],true_loc,w_trim,position_weight)
trimmed_lkd = findlkd_no_constraint(np.sum(sel_ind),caldist(cand_loc[0,sel_ind,:],bike_loc,bike_num),\
                                    beta0,np.repeat(beta1_true,np.sum(sel_ind)).reshape(-1,1),w_trim,
                                    bike_num,num_records,book_bike,book_index,num_booked,all_period)
bic = -trimmed_lkd+0.5*num_remains*np.log(num_booked)
print('Number of Iterations:', iter)
print('The Wasserstein distance is:',was_dist_disc[0])
print('The log-likelihood value is:',trimmed_lkd)
print('The BIC value is:', bic)

Number of Iterations: 6
The Wasserstein distance is: 3.0082090615720194
The log-likelihood value is: -3353.632824676505
The BIC value is: 3378.311267606471
